# SQL Essentials for Databricks Data Engineering

## Overview
This notebook covers SQL fundamentals and advanced concepts needed for Databricks. We'll explore joins, window functions, CTEs, aggregations, and performance considerations.

## Learning Objectives
- Master SQL joins and subqueries
- Understand window functions
- Use Common Table Expressions (CTEs)
- Apply aggregations and grouping
- Optimize query performance

---

## 1. Basic SQL Review

Let's start with fundamental SQL operations using sample data.

In [ ]:
# In Databricks, you can use %%sql magic command or spark.sql()
# For this notebook, we'll show SQL syntax in comments

# Sample data setup (in Databricks, you'd create actual tables)
sample_queries = """
-- Create sample customers table
CREATE OR REPLACE TEMP VIEW customers AS
SELECT * FROM VALUES
  (1, 'Alice', 'alice@email.com', 'NY', '2023-01-15'),
  (2, 'Bob', 'bob@email.com', 'CA', '2023-02-20'),
  (3, 'Charlie', 'charlie@email.com', 'TX', '2023-03-10'),
  (4, 'Diana', 'diana@email.com', 'NY', '2023-04-05')
AS customers(customer_id, name, email, state, signup_date);

-- Create sample orders table
CREATE OR REPLACE TEMP VIEW orders AS
SELECT * FROM VALUES
  (101, 1, 150.00, '2023-06-01'),
  (102, 1, 200.00, '2023-06-15'),
  (103, 2, 75.50, '2023-06-10'),
  (104, 3, 300.00, '2023-06-20'),
  (105, 1, 120.00, '2023-07-01')
AS orders(order_id, customer_id, amount, order_date);
"""

print("Sample table creation SQL:")
print(sample_queries)

### SELECT and WHERE Clauses

In [ ]:
# Basic SELECT
query_1 = """
-- Select all customers
SELECT customer_id, name, email, state
FROM customers;

-- Select with WHERE clause
SELECT name, email
FROM customers
WHERE state = 'NY';

-- Multiple conditions
SELECT name, state, signup_date
FROM customers
WHERE state IN ('NY', 'CA')
  AND signup_date >= '2023-02-01';
"""

print(query_1)

### Aggregations and GROUP BY

In [ ]:
query_aggregations = """
-- Count orders per customer
SELECT customer_id, COUNT(*) as order_count
FROM orders
GROUP BY customer_id;

-- Sum, average, min, max
SELECT 
  customer_id,
  COUNT(*) as order_count,
  SUM(amount) as total_spent,
  AVG(amount) as avg_order_value,
  MIN(amount) as min_order,
  MAX(amount) as max_order
FROM orders
GROUP BY customer_id
HAVING SUM(amount) > 200;  -- Filter aggregated results

-- Multiple grouping columns
SELECT 
  c.state,
  MONTH(o.order_date) as order_month,
  COUNT(*) as orders,
  SUM(o.amount) as total_revenue
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.state, MONTH(o.order_date)
ORDER BY c.state, order_month;
"""

print(query_aggregations)

## 2. SQL Joins

Understanding joins is crucial for combining data from multiple tables.

In [ ]:
query_joins = """
-- INNER JOIN - only matching rows
SELECT 
  c.customer_id,
  c.name,
  o.order_id,
  o.amount
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id;

-- LEFT JOIN - all customers, even without orders
SELECT 
  c.customer_id,
  c.name,
  o.order_id,
  o.amount
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id;

-- Find customers with no orders
SELECT 
  c.customer_id,
  c.name
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_id IS NULL;

-- FULL OUTER JOIN - all rows from both tables
SELECT 
  c.customer_id,
  c.name,
  o.order_id
FROM customers c
FULL OUTER JOIN orders o ON c.customer_id = o.customer_id;

-- CROSS JOIN - Cartesian product (use carefully!)
SELECT 
  c.name,
  d.date
FROM customers c
CROSS JOIN (
  SELECT DISTINCT order_date as date FROM orders
) d;
"""

print(query_joins)

## 3. Subqueries

Subqueries allow complex filtering and calculations.

In [ ]:
query_subqueries = """
-- Scalar subquery - returns single value
SELECT 
  customer_id,
  amount,
  amount - (SELECT AVG(amount) FROM orders) as amount_vs_avg
FROM orders;

-- Subquery in WHERE clause
SELECT name, email
FROM customers
WHERE customer_id IN (
  SELECT customer_id 
  FROM orders 
  WHERE amount > 150
);

-- Correlated subquery - references outer query
SELECT 
  c.name,
  c.state,
  (
    SELECT COUNT(*) 
    FROM orders o 
    WHERE o.customer_id = c.customer_id
  ) as order_count
FROM customers c;

-- EXISTS - check for existence
SELECT name
FROM customers c
WHERE EXISTS (
  SELECT 1 
  FROM orders o 
  WHERE o.customer_id = c.customer_id 
    AND o.amount > 200
);
"""

print(query_subqueries)

## 4. Common Table Expressions (CTEs)

CTEs improve query readability and enable recursive queries.

In [ ]:
query_ctes = """
-- Simple CTE
WITH high_value_orders AS (
  SELECT customer_id, order_id, amount
  FROM orders
  WHERE amount > 100
)
SELECT 
  c.name,
  h.order_id,
  h.amount
FROM high_value_orders h
JOIN customers c ON h.customer_id = c.customer_id;

-- Multiple CTEs
WITH 
customer_stats AS (
  SELECT 
    customer_id,
    COUNT(*) as order_count,
    SUM(amount) as total_spent
  FROM orders
  GROUP BY customer_id
),
top_customers AS (
  SELECT customer_id
  FROM customer_stats
  WHERE total_spent > 200
)
SELECT 
  c.name,
  cs.order_count,
  cs.total_spent
FROM customers c
JOIN customer_stats cs ON c.customer_id = cs.customer_id
WHERE c.customer_id IN (SELECT customer_id FROM top_customers);

-- CTEs for data transformation pipeline
WITH 
bronze AS (
  -- Raw data extraction
  SELECT * FROM orders
),
silver AS (
  -- Data cleaning and enrichment
  SELECT 
    order_id,
    customer_id,
    amount,
    order_date,
    YEAR(order_date) as order_year,
    MONTH(order_date) as order_month
  FROM bronze
  WHERE amount > 0
),
gold AS (
  -- Business aggregations
  SELECT 
    order_year,
    order_month,
    COUNT(*) as total_orders,
    SUM(amount) as total_revenue,
    AVG(amount) as avg_order_value
  FROM silver
  GROUP BY order_year, order_month
)
SELECT * FROM gold ORDER BY order_year, order_month;
"""

print(query_ctes)

## 5. Window Functions

Window functions perform calculations across rows related to the current row.

In [ ]:
query_window_functions = """
-- ROW_NUMBER - assign unique sequential integers
SELECT 
  customer_id,
  order_date,
  amount,
  ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) as order_sequence
FROM orders;

-- RANK and DENSE_RANK
SELECT 
  customer_id,
  amount,
  RANK() OVER (ORDER BY amount DESC) as rank,
  DENSE_RANK() OVER (ORDER BY amount DESC) as dense_rank
FROM orders;

-- LAG and LEAD - access previous/next rows
SELECT 
  customer_id,
  order_date,
  amount,
  LAG(amount) OVER (PARTITION BY customer_id ORDER BY order_date) as previous_order_amount,
  LEAD(amount) OVER (PARTITION BY customer_id ORDER BY order_date) as next_order_amount,
  amount - LAG(amount) OVER (PARTITION BY customer_id ORDER BY order_date) as amount_change
FROM orders;

-- Running totals with SUM
SELECT 
  customer_id,
  order_date,
  amount,
  SUM(amount) OVER (
    PARTITION BY customer_id 
    ORDER BY order_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) as running_total
FROM orders;

-- Moving average
SELECT 
  customer_id,
  order_date,
  amount,
  AVG(amount) OVER (
    PARTITION BY customer_id 
    ORDER BY order_date
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) as moving_avg_3_orders
FROM orders;

-- NTILE - distribute rows into buckets
SELECT 
  customer_id,
  amount,
  NTILE(4) OVER (ORDER BY amount) as quartile
FROM orders;

-- First and last value in partition
SELECT 
  customer_id,
  order_date,
  amount,
  FIRST_VALUE(amount) OVER (
    PARTITION BY customer_id 
    ORDER BY order_date
  ) as first_order_amount,
  LAST_VALUE(amount) OVER (
    PARTITION BY customer_id 
    ORDER BY order_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
  ) as last_order_amount
FROM orders;
"""

print(query_window_functions)

## 6. Advanced SQL Patterns

Common patterns used in data engineering.

In [ ]:
query_advanced = """
-- CASE statement for conditional logic
SELECT 
  customer_id,
  amount,
  CASE 
    WHEN amount < 100 THEN 'Low'
    WHEN amount BETWEEN 100 AND 200 THEN 'Medium'
    ELSE 'High'
  END as order_category
FROM orders;

-- PIVOT - rotate rows to columns
SELECT *
FROM (
  SELECT 
    c.state,
    MONTH(o.order_date) as month,
    o.amount
  FROM customers c
  JOIN orders o ON c.customer_id = o.customer_id
)
PIVOT (
  SUM(amount)
  FOR month IN (6 as June, 7 as July)
);

-- UNION / UNION ALL - combine result sets
SELECT customer_id, 'High Value' as segment
FROM orders
WHERE amount > 200
UNION ALL
SELECT customer_id, 'Regular' as segment
FROM orders
WHERE amount <= 200;

-- INTERSECT - common rows between queries
SELECT customer_id FROM orders WHERE amount > 150
INTERSECT
SELECT customer_id FROM customers WHERE state = 'NY';

-- EXCEPT - rows in first query but not second
SELECT customer_id FROM customers
EXCEPT
SELECT customer_id FROM orders;
"""

print(query_advanced)

## 7. Date and Time Functions

Essential for time-based analysis and partitioning.

In [ ]:
query_date_functions = """
-- Extract date parts
SELECT 
  order_date,
  YEAR(order_date) as year,
  MONTH(order_date) as month,
  DAY(order_date) as day,
  DAYOFWEEK(order_date) as day_of_week,
  QUARTER(order_date) as quarter,
  WEEKOFYEAR(order_date) as week_number
FROM orders;

-- Date arithmetic
SELECT 
  order_date,
  DATE_ADD(order_date, 7) as due_date,
  DATE_SUB(order_date, 1) as previous_day,
  DATEDIFF(CURRENT_DATE(), order_date) as days_since_order
FROM orders;

-- Date formatting
SELECT 
  order_date,
  DATE_FORMAT(order_date, 'yyyy-MM') as year_month,
  DATE_FORMAT(order_date, 'yyyy-MM-dd') as formatted_date,
  DATE_FORMAT(order_date, 'EEEE') as day_name
FROM orders;

-- Truncate dates
SELECT 
  order_date,
  DATE_TRUNC('MONTH', order_date) as month_start,
  DATE_TRUNC('YEAR', order_date) as year_start,
  DATE_TRUNC('WEEK', order_date) as week_start
FROM orders;

-- Current date/time functions
SELECT 
  CURRENT_DATE() as today,
  CURRENT_TIMESTAMP() as now,
  UNIX_TIMESTAMP() as epoch_seconds,
  FROM_UNIXTIME(1234567890) as from_epoch;
"""

print(query_date_functions)

## 8. String Functions

For data cleaning and transformation.

In [ ]:
query_string_functions = """
-- String manipulation
SELECT 
  name,
  UPPER(name) as upper_name,
  LOWER(name) as lower_name,
  LENGTH(name) as name_length,
  CONCAT(name, ' - ', state) as name_state,
  SUBSTRING(email, 1, POSITION('@' IN email) - 1) as username
FROM customers;

-- String cleaning
SELECT 
  TRIM('  spaces  ') as trimmed,
  LTRIM('  left spaces') as left_trim,
  RTRIM('right spaces  ') as right_trim,
  REPLACE('old-value', '-', '_') as replaced;

-- Pattern matching
SELECT name
FROM customers
WHERE name LIKE 'A%'  -- Starts with A
   OR email LIKE '%@%.com';  -- Email format

-- Regular expressions (Spark SQL)
SELECT 
  email,
  REGEXP_EXTRACT(email, '([^@]+)', 1) as username,
  REGEXP_REPLACE(email, '@.*', '@company.com') as normalized_email
FROM customers;

-- Split strings
SELECT 
  SPLIT('2024-06-15', '-') as date_parts,
  SPLIT('tag1,tag2,tag3', ',') as tags;
"""

print(query_string_functions)

## 9. Performance Optimization

Best practices for efficient SQL queries.

In [ ]:
optimization_tips = """
PERFORMANCE OPTIMIZATION TIPS:

1. PARTITIONING
   - Partition tables by frequently filtered columns (date, region, etc.)
   - Query: WHERE date >= '2024-01-01' will skip irrelevant partitions

2. AVOID SELECT *
   - Only select columns you need
   - Reduces data transfer and memory usage
   
3. FILTER EARLY
   - Apply WHERE clauses before joins when possible
   - Reduces data volume in subsequent operations

4. USE APPROPRIATE JOINS
   - Broadcast join for small tables (<10GB in Spark)
   - Sort-merge join for large tables

5. LIMIT SUBQUERIES
   - Use CTEs for better readability and optimization
   - Avoid correlated subqueries when possible

6. AGGREGATION OPTIMIZATION
   - Push down aggregations where possible
   - Use approximate functions for large datasets (approx_count_distinct)

7. CACHE FREQUENTLY USED DATA
   - In Databricks: CACHE TABLE table_name
   - Useful for iterative queries

8. EXPLAIN PLAN
   - Use EXPLAIN to understand query execution
   - Identify bottlenecks and optimization opportunities
"""

print(optimization_tips)

# Example: Explain plan
explain_query = """
EXPLAIN EXTENDED
SELECT 
  c.state,
  COUNT(*) as customer_count,
  SUM(o.amount) as total_revenue
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.state;
"""

print("\nEXPLAIN example:")
print(explain_query)

## 10. Databricks-Specific SQL Features

SQL features unique to Databricks/Delta Lake.

In [ ]:
databricks_sql = """
-- Delta Lake time travel
SELECT * FROM customers VERSION AS OF 10;  -- Version number
SELECT * FROM customers TIMESTAMP AS OF '2024-06-01';  -- Timestamp

-- MERGE (Upsert operation)
MERGE INTO customers target
USING updates source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *;

-- Delta table optimization
OPTIMIZE customers
ZORDER BY (state, signup_date);

-- Vacuum old files
VACUUM customers RETAIN 168 HOURS;  -- 7 days

-- Show table history
DESCRIBE HISTORY customers;

-- Table statistics
DESCRIBE DETAIL customers;

-- Analyze table for optimization
ANALYZE TABLE customers COMPUTE STATISTICS;

-- Create table with properties
CREATE TABLE IF NOT EXISTS sales (
  sale_id BIGINT,
  customer_id BIGINT,
  amount DECIMAL(10,2),
  sale_date DATE
)
USING DELTA
PARTITIONED BY (sale_date)
LOCATION '/mnt/delta/sales'
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
);
"""

print(databricks_sql)

## Practice Exercises

### Exercise 1: Customer Lifetime Value
Calculate total orders, total spent, and average order value per customer.

In [ ]:
# Exercise 1 - Write SQL here
exercise_1 = """
-- TODO: Write a query to calculate:
-- 1. Number of orders per customer
-- 2. Total amount spent per customer
-- 3. Average order value per customer
-- 4. Include customer name from customers table
-- 5. Order by total spent descending

"""

print(exercise_1)

### Exercise 2: Month-over-Month Growth
Calculate monthly revenue and month-over-month growth percentage.

In [ ]:
# Exercise 2 - Write SQL here
exercise_2 = """
-- TODO: Write a query to calculate:
-- 1. Total revenue per month
-- 2. Previous month's revenue using LAG
-- 3. Month-over-month growth percentage
-- Hint: Use window functions and date functions

"""

print(exercise_2)

### Exercise 3: Customer Segmentation
Segment customers into quartiles based on total spending.

In [ ]:
# Exercise 3 - Write SQL here
exercise_3 = """
-- TODO: Write a query to:
-- 1. Calculate total spending per customer
-- 2. Use NTILE to create 4 quartiles
-- 3. Assign segment labels (Premium, Gold, Silver, Bronze)
-- 4. Include customer details

"""

print(exercise_3)

## Summary

In this notebook, you learned:

✅ SQL fundamentals (SELECT, WHERE, GROUP BY, HAVING)
✅ All types of joins (INNER, LEFT, RIGHT, FULL, CROSS)
✅ Subqueries and correlated subqueries
✅ Common Table Expressions (CTEs)
✅ Window functions (ROW_NUMBER, RANK, LAG, LEAD, running totals)
✅ Advanced patterns (CASE, PIVOT, UNION, INTERSECT, EXCEPT)
✅ Date/time and string functions
✅ Performance optimization techniques
✅ Databricks/Delta Lake specific SQL features

## Next Steps

1. Complete the practice exercises
2. Try these queries in Databricks SQL Editor
3. Move to [03-Data-Engineering-Concepts.ipynb](./03-Data-Engineering-Concepts.ipynb)

## Additional Resources

- [Databricks SQL Reference](https://docs.databricks.com/sql/language-manual/index.html)
- [Delta Lake SQL Reference](https://docs.delta.io/latest/delta-batch.html)
- [Spark SQL Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)